### 🔵nn.Sequential 容器

一共有两种形式，接收两种参数：

#### 🔹`torch.nn.Sequential(*args: Module)`

#### 🔹`torch.nn.Sequential(arg: OrderedDict[str, Module])`
一个顺序容器。模块将按照它们在构造函数中传递的顺序添加到其中。另外，也可以传入一个模块的有序字典。`Sequential` 的 `forward()` 方法接受任何输入，并将其转发到其包含的第一个模块。然后它将输出“链式”连接到每个后续模块的输入，最终返回最后一个模块的输出。

与手动调用一系列模块相比，`Sequential` 的价值在于它允许将整个容器视为单个模块，这样对 `Sequential` 进行的转换将应用于其存储的每个模块（每个模块都是 `Sequential` 的一个注册子模块）。

`Sequential` 和 `torch.nn.ModuleList` 之间有什么区别？`ModuleList` 正如其名——用于存储模块的列表！另一方面，`Sequential` 中的层以级联方式连接。
```python
# 使用Sequential创建一个小模型。当运行`model`时，
# 输入首先会传递给`Conv2d(1,20,5)`。`Conv2d(1,20,5)`的输出
# 将用作第一个`ReLU`的输入；第一个`ReLU`的输出将成为
# `Conv2d(20,64,5)`的输入。最后，`Conv2d(20,64,5)`的输出
# 将用作第二个`ReLU`的输入
model = nn.Sequential(
          nn.Conv2d(1,20,5),
          nn.ReLU(),
          nn.Conv2d(20,64,5),
          nn.ReLU()
        )

# 使用有序字典与Sequential。这在功能上与上述代码相同
model = nn.Sequential(OrderedDict([
          ('conv1', nn.Conv2d(1,20,5)),
          ('relu1', nn.ReLU()),
          ('conv2', nn.Conv2d(20,64,5)),
          ('relu2', nn.ReLU())
        ]))
```
##### 🔺`.add(module)`添加单层
```python
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
```

##### 🔺`.append(module)`将给定模块追加到末尾
用法比较简单，就是追加模块的：
```python
model.append(nn.Module)
```
---

### 🔵nn.Embedding

##### 🔺`torch.nn.Embedding(num_embeddings, embedding_dim, padding_idx=None, max_norm=None, norm_type=2.0, scale_grad_by_freq=False, sparse=False, _weight=None, _freeze=False, device=None, dtype=None)`嵌入层
一个简单的查找表，用于存储固定字典和大小的嵌入。此模块经常用于存储词嵌入并使用索引检索它们。模块的输入是一个索引列表，输出是相应的词嵌入。
<br>[形象化解释](https://zhuanlan.zhihu.com/p/647536930)
<br>[Embedding层作用](https://www.jiqizhixin.com/articles/2019-03-27-7)
> 在实现上，用lookup查表来代替矩阵乘积以提高性能

属性:
-**_@attr `.weight`_**<br>
模块的可学习权重，形状为`(num_embeddings, embedding_dim)`，从正态分布 $N(0,1)$ 初始化

常用参数:
- **_@param `num_embeddings (int)`_**<br>
嵌入字典的大小(词的最大个数)
- **_@param `embedding_dim (int)`_**<br>
每个嵌入向量的大小(词向量的维度)

其他参数:
- **_@param `padding_idx (int, optional)`_**<br>
如果指定，则在padding_idx的条目不会对梯度有贡献；因此，训练期间不会更新padding_idx的嵌入向量，即它保持为固定的"填充"。<br>对于新构造的Embedding，padding_idx的嵌入向量默认为全零，但可以更新为另一个值用作填充向量。<br>比如设置padding_idx=0，则任何包含[*,*,0]的最后一个，都会获取一个不更新梯度始终为0的嵌入向量
- **_@param `max_norm (float, optional)`_**<br>
如果给定，范数大于max_norm的每个嵌入向量将被重新标准化以具有max_norm范数
- **_@param `norm_type (float, optional)`_**<br>
用于计算max_norm选项的p-范数的p值。默认为2
- **_@param `scale_grad_by_freq (bool, optional)`_**<br>
如果给定，将按照小批次中单词的频率的倒数来缩放梯度,默认为False
- **_@param `sparse (bool, optional)`_**<br>
如果为True，权重矩阵相对于的梯度将是一个稀疏张量

形状:
- 输入: `(∗)`，任意形状的IntTensor或LongTensor，包含要提取的「索引」
- 输出: `(∗, H)`，其中 `*` 是输入形状，`H=embedding_dim`

> 注意:当`max_norm`不为None时，Embedding的前向方法将修改权重张量就地。由于用于梯度计算的张量不能就地修改，在调用Embedding的前向方法之前对`Embedding.weight`执行可微操作需要克隆`Embedding.weight`


类方法:

##### 🔺**_@method `from_pretrained(embeddings, freeze=True, padding_idx=None, max_norm=None, norm_type=2.0, scale_grad_by_freq=False, sparse=False)`_**
从给定的二维FloatTensor创建Embedding实例

特殊参数:
- **_@param `embeddings (Tensor)`_**<br>
包含Embedding权重的FloatTensor。第一维被传递给Embedding作为num_embeddings，第二维作为embedding_dim
- **_@param `freeze (bool, optional)`_**<br>
如果为True，则张量在学习过程中不会更新。相当于embedding.weight.requires_grad = False。默认值：True

相同参数:
- **_@param `padding_idx (int, optional)`_**<br>
- **_@param `max_norm (float, optional)`_**<br>
- **_@param `norm_type (float, optional)`_**<br>
- **_@param `scale_grad_by_freq (bool, optional)`_**<br>
- **_@param `sparse (bool, optional)`_**<br>


代码示例:<br>
常常用作预训练的FloatTensor:
```python
weight = torch.FloatTensor([[1, 2.3, 3], [4, 5.1, 6.3]])
embedding = nn.Embedding.from_pretrained(weight)
...
embedding(input)
```


##### 附录：
1. 嵌入向量的范数<br>
    嵌入向量的范数是指向量的大小或长度。在数学上，对于一个给定的向量 $\mathbf{v}$，其范数为：

    $$\| \mathbf{v} \| = \sqrt{v_1^2 + v_2^2 + \dots + v_n^2}$$

    其中 $v_1, v_2, \dots, v_n$ 是向量 $\mathbf{v}$ 的各个分量。这就是所谓的 $L_2$ 范数或欧几里得范数。

    例如，对于向量 $\mathbf{v} = [3, 4]$，其范数为 5，这是因为 $\sqrt{3^2 + 4^2} = 5$。

    在深度学习和词嵌入的上下文中，范数常常用来度量一个向量的大小。通过限制嵌入向量的范数，我们可以防止它们变得太大，这可能会导致模型的不稳定。

2. `padding_idx`用途：<br>
    在处理可变长度的词的时候，通常需要进行填充到统一长度，比如:<br>
    A[3, 5, 1], B[2, 4], C[7]<br>
    为了处理这些句子，会将填充到长度为3:
    A[3, 5, 1], B[2, 4, 0], C[7, 0, 0]<br>
    这时候就要指定`padding_idx=0`，那么索引0对应的嵌入向量将始终为全零，并且在训练过程中不会被更新

3. 如何修改`padding_idx`对应的权重：
    ```python
    with torch.no_grad():
        embedding.weight[padding_idx] = torch.ones(3)
    ```

代码示例:
如下你将看到一个(2,4,3)shape的tensor
```python
embedding = nn.Embedding(10, 3)
input = torch.LongTensor([[1, 2, 4, 5], [4, 3, 2, 9]])
embedding(input)
```

---